In [4]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import cv2
from keras.models import load_model
print(tf.__version__)


2.7.0


In [5]:
# hàm nomarlize data
def preprocessing(img):
    img=img.astype("uint8")
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert sang màu xám để tăng hiệu năng thay vì để 3 màu
    img=cv2.equalizeHist(img) # cân bằng biểu đồ xám, cải thiện độ tương phản hình ảnh, mở rộng phạm vi cường độ
    img = img/255
    return img
# lấy tên của nhãn dán
def get_className(classNo):
	if classNo==0:
		return "Mask"
	elif classNo==1:
		return "No Mask"


In [2]:
# sử dụng file haarcascade_frontalface_default.xml đã tích hợp sẵn thuật toán)
# File này sẽ tích hợp sẵn các feature để nhận diện khuôn mặt
# Haar Cascade classifier ~ effective approach detection object
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

threshold=0.90

# khởi tạo camera đầu tiên của máy tính, có index = 0
cap=cv2.VideoCapture(0)

# set parameter cho camera
cap.set(3, 640)
cap.set(4, 480)

font=cv2.FONT_HERSHEY_COMPLEX

#load model
model = load_model('MyTrainingModel.h5')

NameError: name 'cv2' is not defined

In [1]:
while True:
    sucess, imgOrignal=cap.read()
    # sucess có kiểu là boolean, dòng này có nghĩa nếu imgOrignal được đọc thành công nếu sucess = true(tức mở camera thành công)
    # khá là giống success&imgOrignal= cap.read()
    
    # trả về hình chữ nhật ranh giới cho khuôn mặt được phát hiện
    # có ý nghĩa sẽ dùng mọi feature của file haarcascade_frontalface_default.xml để phát hiện
    faces = facedetect.detectMultiScale(imgOrignal,1.3,5) #image/size image được giảm/
    # ở đây ta giảm tỉ lệ size của ảnh đi 1,3 lần để trong trường hợp khuôn mặt to sẽ giảm đi để có thể detect được khuôn mặt
    # điều này có thể khiến việc detect nhanh hơn, thuật toán chạy nhanh hơn tuy nhiên
    # Khi giảm tỉ lệ đi đồng nghĩa có thể bỏ sót 1 vài chi tiết của khuôn mặt do vậy không thể giảm quá nhiều
    # 5 là tham số chỉ đỉnh số lượng hàng xóm mà hình chữ nhật khuông mặt có(kiểu hình chữ nhật nhỏ xung quanh ấy :)) cũng không chắc nữa). Nó sẽ quyết định
    # đến chất lượng khi phát hiện ảnh. Giá trị càng cao thì ít bị phát hiện hơn nhưng chất lượng cao hơn. 5 là con số vừa đủ
    
    for x,y,w,h in faces: # x-cor, y-cor, width, height, dựa vào 4 tham số này để vẽ hình chữ nhật xung quanh khuôn mặt 
        crop_img=imgOrignal[y:y+h,x:x+w] # lấy ảnh chứa mặt, từ hàng y-> y+h, từ cột x->x+w
        
        # chuyển sang kênh màu xám vì kênh màu xám dễ xử lý hơn, và ít chuyên sâu về tính toán vì nó chỉ chứa 1 kênh đen trắng
        # resize 32x32
        img=cv2.resize(crop_img, (32,32)) 
        
        # normalize img
        img=preprocessing(img)
        
        # reshape lại
        img=img.reshape(1, 32, 32, 1) # 1 ảnh/32/32/kênh gray

        # tính prediction từ model
        prediction=model.predict(img)
        
        # lấy classIndex của prediction, prediction có dạng là một numpy array, np.argmax trả về index của giá trị lớn nhất
        # của mảng bắt đầu theo cột chỉ định là 1
        classIndex= np.argmax(prediction,axis=1)
        
        probabilityValue=np.amax(prediction) #trả về giá trị lớn nhất
        print(probabilityValue, " " , classIndex," ", prediction)
        if probabilityValue<threshold:
#             print(classIndex)
            if classIndex==1:
#                # vẽ giao diện
                cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(0,255,0),2) # điểm bắt đầu, điểm kết thúc (2 điểm của 2 góc hình chữ nhật), màu, độ dày của cạnh
                cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (0,255,0),-2)
                cv2.putText(imgOrignal, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
            elif classIndex==0:
#              
                cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(50,50,255),2)
                cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (50,50,255),-2)
                cv2.putText(imgOrignal, str(get_className(classIndex)),(x,y-10), font, 0.75, (255,255,255),1, cv2.LINE_AA)
    cv2.imshow("Result",imgOrignal)
    
    k=cv2.waitKey(1) #đợi nhập key
    if k==ord('q'):# nhập q để break loop
        break


cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined